## Dado que algunas paginas no permiten scrapear...

Vamos a utilizar beatifulSoap para poder hacerlo.

### Primera versión

Extracción de noticias del diario el cronista, utilizando fuentes RSS.

Al final se indica la cantidad de dastos extraidos

In [3]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URL de la fuente RSS
url = 'https://www.cronista.com/files/rss/news.xml'

# Definir encabezados HTTP para simular una solicitud de navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Intentar obtener los datos de la fuente RSS con los encabezados
response = requests.get(url, headers=headers)
response.raise_for_status()  # Asegurarse que la solicitud fue exitosa

# Parsear el contenido usando lxml-xml como parser
soup = bs(response.content, 'lxml-xml')

# Encontrar todos los elementos 'item'
items = soup.find_all('item')

data = []

for item in items:
    title = item.find('title').text if item.find('title') else None
    link = item.find('link').next_sibling.strip() if item.find('link') else None
    description = item.find('description').text if item.find('description') else None
    pub_date = item.find('pubDate').text if item.find('pubDate') else None
    

    # Añade los datos extraídos a la lista como un diccionario
    data.append({
        'Title': title,
        'Link': link,
        'Description': description,
        'Publication Date': pub_date
        })


# Crea el DataFrame
df = pd.DataFrame(data)

# Muestra el DataFrame
print(df.head())


print(f"Cantidad de registros: {len(items)}")


                                               Title Link  \
0  Volvió a subir el dólar y el BCRA compró el mo...        
1  Dieta Keto: los alimentos y bebidas que tenés ...        
2  Macri asumió la presidencia del PRO: cuál fue ...        
3  Dólar blue HOY: a cuánto cerró la cotización e...        
4  Aumenta el subte: la justicia porteña dio marc...        

                                         Description  \
0  Los dólares financieros avanzaron hasta 1% y e...   
1  Conocer los alimentos prohibidos en una dieta ...   
2  El exmandatario volvió a la conducción formal ...   
3  Estos son los datos más importantes a los que ...   
4  La jueza Elena Liberatori recibió la informaci...   

                Publication Date  
0  Thu, 16 May 2024 22:02:00 GMT  
1  Thu, 16 May 2024 22:00:00 GMT  
2  Thu, 16 May 2024 21:45:00 GMT  
3  Thu, 16 May 2024 21:08:27 GMT  
4  Thu, 16 May 2024 20:58:35 GMT  
Cantidad de registros: 100


### Versión 2
Similar a la anterior, solo que se suma la URL de la noticia



In [15]:
# Usar comprensión de lista para crear una lista de diccionarios con la información de cada ítem
articles = [{
    'title': item.find('title').text if item.find('title') else "No title provided",
    'link': item.find('link').text.strip() if item.find('link') and item.find('link').text else "No link provided",
    'description': item.find('description').text if item.find('description') else "No description provided",
    'pub_date': item.find('pubDate').text if item.find('pubDate') else "No publication date provided"
} for item in items]


# Añade los datos extraídos a la lista como un diccionario
data.append({
    'Title': title,
    'Link': link,
    'Description': description,
    'Publication Date': pub_date
    })

# Crea el DataFrame
df = pd.DataFrame(data)

# Muestra el DataFrame
df.head()

# Imprimir la cantidad de registros
print(f"Cantidad de registros: {len(articles)}")


Title: El Gobierno reducirá los aranceles a la importación de heladeras, lavarropas y neumáticos
Link: https://www.cronista.com/economia-politica/el-gobierno-redujo-los-aranceles-a-la-importacion-de-heladeras-lavarropas-y-neumaticos/
Description: Lo confirmó el vocero Manuel Adorni. "Esto implica mejores precios y un mayor nivel de competencia en el mercado", precisó.
Publication Date: Sat, 04 May 2024 12:46:50 GMT

Title: España rechazó el duro comunicado del Milei: "No se corresponde con las relaciones de dos países hermanos"
Link: https://www.cronista.com/economia-politica/espana-rechazo-el-duro-comunicado-del-gobierno-argentino-no-se-corresponde-con-las-relaciones-de-dos-paises-hermanos/
Description: El Ministerio de Asuntos Exteriores de la nación europea condenó los términos utilizados en la misiva y  remarcó que seguirán "manteniendo y fortaleciendo sus lazos fraternales con el pueblo argentino".
Publication Date: Sat, 04 May 2024 12:30:00 GMT

Title: Expectativa por inversiones

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_news(urls):
    data = []  # Lista para almacenar todos los datos recopilados

    for url in urls:
        try:
            # Realizar la petición HTTP a la página
            response = requests.get(url)
            response.raise_for_status()  # Asegurar que la respuesta es exitosa

            # Parsear el contenido HTML de la página
            soup = BeautifulSoup(response.text, 'html.parser')

            # Encontrar todos los elementos que contengan información de artículos
            items = soup.find_all('article')

            # Extraer datos de cada artículo
            articles = [{
                'newspaper': url.split('/')[2],  # Extraer el nombre del dominio como nombre del diario
                'title': item.find('h2').get_text(strip=True) if item.find('h2') else "No title provided",
                'link': item.find('a')['href'] if item.find('a') else "No link provided",
                'description': item.find('p').get_text(strip=True) if item.find('p') else "No description provided",
                'pub_date': item.find('time').get_text(strip=True) if item.find('time') else datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
            } for item in items]

            # Agregar los artículos de esta página al listado general
            data.extend(articles)

        except Exception as e:
            print(f"Error scraping {url}: {e}")
            continue

    # Convertir la lista de diccionarios a DataFrame
    df = pd.DataFrame(data)
    return df

# Ejemplo de uso:
urls = [
    'https://www.lanacion.com.ar/economia/',
    'https://www.ambito.com/economia',
    'https://www.cronista.com/Seccion/Economia/',
    'https://eleconomista.com.ar/',
    'https://www.infobae.com/economia/',
    'https://www.forbesargentina.com/temas/finanzas-t17'
]

news_df = scrape_news(urls)
print(news_df)


Error scraping https://www.cronista.com/Seccion/Economia/: 403 Client Error: Forbidden for url: https://www.cronista.com/Seccion/Economia/
                   newspaper  \
0        www.lanacion.com.ar   
1        www.lanacion.com.ar   
2        www.lanacion.com.ar   
3        www.lanacion.com.ar   
4        www.lanacion.com.ar   
..                       ...   
170  www.forbesargentina.com   
171  www.forbesargentina.com   
172  www.forbesargentina.com   
173  www.forbesargentina.com   
174  www.forbesargentina.com   

                                                 title  \
0    Martín Redrado.“Para darle credibilidad a la p...   
1    ¿Todos a los botes?Se acelera el debate por la...   
2            Análisis.La Dama y el León (y la lechuga)   
3    Debate.El impuesto a los ingresos netos debe s...   
4    Movilidad.Qué aumentos para las jubilaciones s...   
..                                                 ...   
170  La empresa que debutó en la Bolsa y sus accion...   
171  Warner 

In [9]:
news_df

,newspaper,title,link,description,pub_date
0,www.lanacion.com.ar,Martín Redrado.“Para darle credibilidad a la p...,/economia/martin-redrado-para-darle-credibilid...,No description provided,2024-05-12 22:14:01
1,www.lanacion.com.ar,¿Todos a los botes?Se acelera el debate por la...,/economia/del-efecto-guau-al-efecto-todos-a-lo...,No description provided,2024-05-12 22:14:01
2,www.lanacion.com.ar,Análisis.La Dama y el León (y la lechuga),/economia/la-dama-y-el-leon-y-la-lechuga-nid12...,No description provided,2024-05-12 22:14:01
3,www.lanacion.com.ar,Debate.El impuesto a los ingresos netos debe s...,/economia/el-impuesto-a-los-ingresos-netos-deb...,No description provided,2024-05-12 22:14:01
4,www.lanacion.com.ar,Movilidad.Qué aumentos para las jubilaciones s...,/economia/jubilaciones-que-dicen-los-proyectos...,No description provided,2024-05-12 22:14:01
...,...,...,...,...,...
170,www.forbesargentina.com,La empresa que debutó en la Bolsa y sus accion...,https://www.forbesargentina.com/money/la-empre...,La OPI de HD Hyundai Marine Solution es la may...,2024-05-12 22:14:05
171,www.forbesargentina.com,Warner Bros puede perder la NBA pero sus accio...,https://www.forbesargentina.com/money/warner-b...,Los inversores destrozaron las acciones del gi...,2024-05-12 22:14:05
172,www.forbesargentina.com,Argentina en el radar de Elon Musk: ¿Un nuevo ...,https://www.forbesargentina.com/money/argentin...,"Además del multimillonario sudafricano, hubo o...",2024-05-12 22:14:05
173,www.forbesargentina.com,"El dividendo con ganancias de hasta 9,3% en dó...",https://www.forbesargentina.com/money/el-divid...,Una tendencia inusual llegó a Silicon Valley m...,2024-05-12 22:14:05


### Versión final

Trabajaremos sobre noticias de indole economica, utilizaremos la fuentes RSS, por disponer de mas contenido a diferencia de la extracción de titulos y subtitulos.

Pasos:

1 - Buscar fuentes de diarios que traten noticias economicas y extracción de esos datos. Ok

2 - Generar el proceso de extracción 1 vez al día. OK -  Analizar la posibilidad de realizarlo en automatico

3 - Generar un analisis de sentimiento usando libreria de huggings face - 

4 - Generar una clasificación de temas en politica economica e industrias emergentes y mercado agro.

### 1- Fuente de datos.

Se podrá trabajar con las fuentes RSS de economia internacionales y nacionales

#### Internacionales
https://es.mercopress.com/rss/

https://www.oecd.org/index.xml

https://www.cnbc.com/id/20910258/device/rss

https://feeds.washingtonpost.com/rss/business?itid=lk_inline_manual_36

https://www.reutersagency.com/feed/?taxonomy=best-sectors&post_type=best

https://www.federalreserve.gov/feeds/press_all.xml

https://www.bbc.com/mundo/temas/economia/index.xml


#### Nacionales
https://www.perfil.com/feed/economia

https://www.clarin.com/rss/economia/

https://www.ambito.com/rss/pages/economia.xml

https://www.ambito.com/rss/pages/negocios.xml

https://www.iprofesional.com/rss/economia


### Extracción de fuente de datos

#### Función para extraer los datos co manejo de excepción

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime

def fetch_rss_data(rss_urls):
    all_data = []
    failed_sources = []  # Lista para almacenar las fuentes que fallan
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    for url in rss_urls:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Asegura que la solicitud fue exitosa
            
            soup = bs(response.content, 'lxml-xml')
            diario = url.split('/')[2]  # Extraer nombre del diario del dominio de la URL

            items = soup.find_all('item')
            for item in items:
                title = item.find('title').text if item.find('title') else None
                description = item.find('description').text if item.find('description') else None
                pub_date = item.find('pubDate').text if item.find('pubDate') else datetime.now().strftime("%a, %d %b %Y %H:%M:%S %z")
                
                all_data.append({
                    'diario': diario,
                    'titulo': title,
                    'contenido': description,
                    'fecha de extracción': pub_date
                })
        
        except Exception as e:
            print(f"Falla en el proceso RSS desde: {url}, Error: {str(e)}")
            failed_sources.append(url)  # Agregar URL a la lista de fuentes fallidas

    df = pd.DataFrame(all_data)
    return df, failed_sources

#### Limpieza de html

Se analiza la estructura de los html para decidir que es lo que se requiere limpiar.


In [3]:
from bs4 import BeautifulSoup

def clean_html(content):
    # Verificar si el contenido es None
    if content is None:
        return ""
    
    # Parsear el contenido usando BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')
    
    # Eliminar etiquetas indeseadas y sus contenidos
    for tag in soup(['script', 'noscript', 'iframe']):
        tag.decompose()  # Elimina la etiqueta y su contenido

    # Eliminar etiquetas img pero conservar el texto restante
    for tag in soup.find_all('img'):
        tag.decompose()

    # Eliminar etiquetas vacías que pueden haber quedado
    for tag in soup(['p', 'li']):
        tag.unwrap()
    
    # Devolver el texto limpio
    return soup.get_text(strip=True)  # strip=True elimina espacios adicionales


#### Noticias Nacionales

In [9]:
# URLs de fuentes RSS de diarios nacionales
rss_urls_nacionales = ['https://www.ambito.com/rss/pages/negocios.xml',
            'https://www.perfil.com/feed/economia',
            'https://www.clarin.com/rss/economia/',
            'https://www.ambito.com/rss/pages/economia.xml',
            'https://www.ambito.com/rss/pages/negocios.xml',
            'https://www.iprofesional.com/rss/economia']

# Extracción
dataframe_nacionales, failed_sources_nacionales = fetch_rss_data(rss_urls_nacionales)

# Limpieza
dataframe_nacionales['contenido_depurado'] = dataframe_nacionales['contenido'].apply(clean_html)

# Visualización
print(dataframe_nacionales)
if failed_sources_nacionales:
    print("Falla de extracción en la/s fueste/s:", failed_sources_nacionales)



                   diario                                             titulo  \
0          www.ambito.com  La actividad industrial cayó 11,7% en el prime...   
1          www.ambito.com  Cuáles son los barrios con más oferta de inmue...   
2          www.ambito.com  Recesión: las ventas de informática cayeron 33...   
3          www.ambito.com  Quién es Lorinc Mészáros, el humilde instalado...   
4          www.ambito.com  Mediam by Aleph es el representante oficial de...   
..                    ...                                                ...   
163  www.iprofesional.com  El BBVA y el Banco Galicia también lanzaron su...   
164  www.iprofesional.com  La Ley de Bases permitirá captar más de u$s900...   
165  www.iprofesional.com  El banco JP Morgan pronosticó cómo seguirá la ...   
166  www.iprofesional.com  Cuándo el precio del dólar puede volver a subi...   
167  www.iprofesional.com  En el Congreso, Nicolás Posse defendió el plan...   

                                       

#### Noticias internacionales

In [10]:

rss_urls_internacionales = ['https://es.mercopress.com/rss/',
                            'https://www.oecd.org/economy/index.xml',
                            'https://www.cnbc.com/id/20910258/device/rss',
                            'https://feeds.washingtonpost.com/rss/business?itid=lk_inline_manual_36',
                            'https://www.reutersagency.com/feed/?taxonomy=best-sectors&post_type=best',
                            'https://www.federalreserve.gov/feeds/press_all.xml',
                            'https://www.bbc.com/mundo/temas/economia/index.xml'
                            ]

dataframe_internacionales, failed_sources_internacionales = fetch_rss_data(rss_urls_internacionales)


dataframe_internacionales['contenido_depurado'] = dataframe_internacionales['contenido'].apply(clean_html)


print(dataframe_internacionales)
if failed_sources_internacionales:
    print("Falla de extracción en la/s fueste/s:", failed_sources_internacionales)

Failed to process RSS from: https://www.oecd.org/economy/index.xml, Error: 403 Client Error: Forbidden for url: https://www.oecd.org/economy/index.xml
Failed to process RSS from: https://www.reutersagency.com/feed/?taxonomy=best-sectors&post_type=best, Error: 403 Client Error: Forbidden for url: https://www.reutersagency.com/feed/?taxonomy=best-sectors&post_type=best
                    diario                                             titulo  \
0        es.mercopress.com    Producción de carne sigue mejorando en Paraguay   
1        es.mercopress.com  Argentina se prepara para dolarizar su economí...   
2        es.mercopress.com  Futuro europeo de Gibraltar podría afectar est...   
3        es.mercopress.com  Caen acciones de Petrobras tras la salida de P...   
4        es.mercopress.com  Venezuela niega salvoconductos a solicitantes ...   
..                     ...                                                ...   
71  www.federalreserve.gov              Federal Reserve issues 

#### Guardamos los datos con el siguiente formato de salida
fecha_nombre_del_archivo.csv

In [13]:
import pandas as pd
from datetime import datetime
import os

# Obtener la fecha actual en formato de año-mes-día
today = datetime.now().strftime("%Y-%m-%d")

# Definir la ruta del directorio donde se guardará el archivo
directory = './datos'

# Verificar si el directorio existe, si no, crearlo
if not os.path.exists(directory):
    os.makedirs(directory)

### Guardar datos

##### Datos nacionales


In [14]:
# Construir la ruta completa del archivo incluyendo la carpeta
filename_nac = f'{directory}/{today}_noticias_economicas_nacionales.csv'

# Guardar el DataFrame en un archivo CSV en la ruta especificada
dataframe_nacionales.to_csv(filename_nac, index=False, encoding='utf-8')

##### Datos internacionales

In [16]:
# Construir la ruta completa del archivo incluyendo la carpeta
filename_inter = f'{directory}/{today}_noticias_economicas_internacionales.csv'

# Guardar el DataFrame en un archivo CSV en la ruta especificada
dataframe_internacionales.to_csv(filename_inter, index=False, encoding='utf-8')

### Envio de los datos a google drive
